# What this note can do?

NOTE: we creaed original imput files by Extract_nongap-sites_on_HXB2_Rhesus_DNA.ipynb, which can be found at /Users/kais/Documents/CODE/HMMER_HIV/note/

- Table of index and mutation strength. 
- From this CSV file, determin Abs resitance mutation. 

In [40]:
#Pkg.add("Pkg"); import Pkg; 
using Pkg
using Distributed
using DelimitedFiles
using Profile    
using StatsBase
using Random
using Statistics
using LinearAlgebra
using Plots
using Printf
using Measures
using CSV
using LaTeXStrings;
using DataFrames
rng = Random.MersenneTwister(1234);

In [41]:
include("../src/basic_MPL.jl");
include("../src/basic.jl")
include("../src/my_HIV_data_cleanign.jl")
include("../src/basic_analysis.jl")
include("../src/non_synonymous_N-glycan.jl") 
include("../src/process_HIV_mutation_for_CSV.jl") 
NUC = ["-", "A", "C", "G", "T"];

# Processes Start

## Retrieve variable and antibody binding sites 

In [14]:
# Keep trucking only the following sites and nucleotides;
# Should include also HXB2 inde for protein and genes. 
hxb2csv = DataFrame(CSV.File("../data/csv/hxb2.csv"));
csv_raw_DH270_binding = DataFrame(CSV.File("../data/csv/DH270_binding.csv"))
csv_raw_DH272_binding = DataFrame(CSV.File("../data/csv/DH272_binding.csv"))
csv_raw_DH475_binding = DataFrame(CSV.File("../data/csv/DH475_binding.csv"))
csv_raw_CD4contact = DataFrame(CSV.File("../data/csv/CD4_contct_residue.csv"))
csv_raw_V1_HV = DataFrame(CSV.File("../data/csv/V1_Hypervariable_residue.csv"))
csv_raw_V2_HV = DataFrame(CSV.File("../data/csv/V2_Hypervariable_residue.csv"))
csv_raw_V3_HV = DataFrame(CSV.File("../data/csv/V3_Hypervariable_residue.csv"))
csv_raw_V4_HV = DataFrame(CSV.File("../data/csv/V4_Hypervariable_residue.csv"))
csv_raw_V5_HV = DataFrame(CSV.File("../data/csv/V5_Hypervariable_residue.csv"))
csv_raw_LoopD = DataFrame(CSV.File("../data/csv/Loop-D_residue.csv"));

csv_raw_resist_mut_DH270 = DataFrame(CSV.File("../data/csv/resistant_mut_to_DH270.csv")) 
csv_raw_resist_mut_DH272 = DataFrame(CSV.File("../data/csv/resistant_mut_to_DH272.csv")) 
csv_raw_resist_mut_DH475 = DataFrame(CSV.File("../data/csv/resistant_mut_to_DH475.csv")) 
csv_raw_resist_strain_specific_Abs_CH848 = DataFrame(CSV.File("../data/csv/resistant_mut_to_strain_specific-bnAbs_CH848.csv")) 
csv_raw_common_mut_SHIV_CH848 = DataFrame(CSV.File("../data/csv/common_mutation_rhesus_SHIV848.csv")) ;

## Formatting the files

In [17]:
#csv_selection = DataFrame(CSV.File("/net/dali/home/barton/kais/paper-MPL-inference-master/data/HIV/analysis/703010848-3-poly.csv"));
csv_selection = DataFrame(CSV.File("../data/mpl/703010848-3-poly.csv"));
gene_set = [[] for _ in 1:3]; vec_AA_idx = [[] for _ in 1:3]; HXB2_set = [[] for _ in 1:3]; 
vec_nuc_idx = []; frame_set = []; 
for x in csv_selection.HXB2_index
    num_nuc = get_num_nuc(x)
    frame, gene = index2frame(num_nuc)
    num_nuc, num_AA, gene = map_numNUC_to_numAA(x, frame)    
    push!(frame_set, frame)
    push!(vec_nuc_idx, num_nuc)
    for fr in 1:3 
        push!(gene_set[fr], gene[fr])
        push!(vec_AA_idx[fr], num_AA[fr])
        push!(HXB2_set[fr], HXB2(num_nuc, num_AA[fr], gene[fr]))
    end
end;

@printf("Fr.1: gene: %s\n", join(unique(gene_set[1]), ", "));
@printf("Fr.2: gene: %s\n", join(unique(gene_set[2]), ", "));
@printf("Fr.3: gene: %s\n", join(unique(gene_set[3]), ", "));
@printf("max: %d, %d\n", maximum(skipmissing(vec_nuc_idx)), maximum(skipmissing(vec_nuc_idx))) 
@printf("min: %d, %d\n", minimum(skipmissing(vec_nuc_idx)), minimum(skipmissing(vec_nuc_idx)));

# Use frame 3 where Env gene belongs. 
vec_DH270_binding = get_true_false_variable_region(vec_AA_idx[3], csv_raw_DH270_binding)
vec_DH272_binding = get_true_false_variable_region(vec_AA_idx[3], csv_raw_DH272_binding)
vec_DH475_binding = get_true_false_variable_region(vec_AA_idx[3], csv_raw_DH475_binding)
vec_CD4contact_binding = get_true_false_variable_region(vec_AA_idx[3], csv_raw_CD4contact)
vec_V1 = get_true_false_variable_region(vec_AA_idx[3], csv_raw_V1_HV)
vec_V2 = get_true_false_variable_region(vec_AA_idx[3], csv_raw_V2_HV)
vec_V3 = get_true_false_variable_region(vec_AA_idx[3], csv_raw_V3_HV)
vec_V4 = get_true_false_variable_region(vec_AA_idx[3], csv_raw_V4_HV)
vec_V5 = get_true_false_variable_region(vec_AA_idx[3], csv_raw_V5_HV)
vec_LoopD = get_true_false_variable_region(vec_AA_idx[3], csv_raw_LoopD);

csv_raw_resist_mut_DH270 = DataFrame(CSV.File("../data/csv/resistant_mut_to_DH270.csv")) 
csv_raw_resist_mut_DH272 = DataFrame(CSV.File("../data/csv/resistant_mut_to_DH272.csv")) 
csv_raw_resist_mut_DH475 = DataFrame(CSV.File("../data/csv/resistant_mut_to_DH475.csv")) 
csv_raw_resist_strain_specific_Abs_CH848 = DataFrame(CSV.File("../data/csv/resistant_mut_to_strain_specific-bnAbs_CH848.csv")) 
csv_raw_common_mut_SHIV_CH848 = DataFrame(CSV.File("../data/csv/common_mutation_rhesus_SHIV848.csv")) ;

# These are all waat I need. 
csv_index_and_TF= DataFrame(CSV.File("../data/mpl/703010848-3-index.csv"));
seq_num_raw = readdlm("../data/mpl/703010848-3-poly-seq2state.dat", Int);

Fr.1: gene: , vif, tat, nef
Fr.2: gene: , tat, vpu, rev
Fr.3: gene: pol, , vpr, rev, env
max: 9603, 9603
min: 4924, 4924


## Processes to get mutations 

In [18]:
(mutant_hxb2, mutant_nuc, mutant_date_found, mutant_gene, mutant_types_set_AA, 
mutant_types_set_nuc, mutant_types_set_nuc_simple, plus_glycan_set, minus_glycan_set, shifted_glycan_set
) = Make_combination_of_mutations_with_genetic_background_w_glycan(csv_index_and_TF, seq_num_raw, hxb2csv); 
mutant_types_set_AA_filtered = []
push!(mutant_types_set_AA_filtered, filtering_mutations_AA(mutant_types_set_AA[1]));
push!(mutant_types_set_AA_filtered, filtering_mutations_AA(mutant_types_set_AA[2]));
push!(mutant_types_set_AA_filtered, filtering_mutations_AA(mutant_types_set_AA[3]));


mutant_types_set_TF_AA = get_TF_AA(csv_index_and_TF, mutant_hxb2) ;
mutant_types_set_AA_filtered_v2 = []
push!(mutant_types_set_AA_filtered_v2, replacing_redundant_AA_by_TF(mutant_types_set_AA_filtered[1], mutant_types_set_TF_AA[1]))
push!(mutant_types_set_AA_filtered_v2, replacing_redundant_AA_by_TF(mutant_types_set_AA_filtered[2], mutant_types_set_TF_AA[2]))
push!(mutant_types_set_AA_filtered_v2, replacing_redundant_AA_by_TF(mutant_types_set_AA_filtered[3], mutant_types_set_TF_AA[3]));
mutant_nuc_simple_filtered = filter_nuc_mut(mutant_types_set_nuc_simple);


i_raw=1 hxb2=4924 a_WT=A
cdn_bfr:ATC
cdn_afr:---
Detected Codon: before>ATC & after>---
Detected AA: before>I & after>-
Possible Transition (row: before, col: after): 
0 
Possible Transition (After): 
0 
fr:3 gene:pol poly_idx:1 HXB2_idx:4924 nuc:- detected:949d
GENE:A4924- AA:I947- NUC:ATC4924-4926---
fr:1 gen:NA mut:NA mut_simple:NA mut_AA:NA
fr:2 gen:NA mut:NA mut_simple:NA mut_AA:NA
fr:3 gen:pol mut:ATC4924-4926--- mut_simple:A4924- mut_AA:I947-
cdn_bfr:ATC
cdn_afr:ATC
Detected Codon: before>ATC & after>ATC
Detected AA: before>I & after>I
Possible Transition (row: before, col: after): 
0 
Possible Transition (After): 
0 
fr:3 gene:pol poly_idx:1 HXB2_idx:4924 nuc:A detected:17d
GENE:A4924A AA:I947I NUC:ATC4924-4926ATC
fr:1 gen:NA mut:NA mut_simple:NA mut_AA:NA
fr:2 gen:NA mut:NA mut_simple:NA mut_AA:NA
fr:3 gen:pol mut:ATC4924-4926ATC mut_simple:A4924A mut_AA:I947I

i_raw=2 hxb2=4925 a_WT=T
cdn_bfr:ATC
cdn_afr:---
Detected Codon: before>ATC & after>---
Detected AA: before>I & afte

cdn_afr:TTT
Detected Codon: before>TTT & after>TTT
Detected AA: before>F & after>F
Possible Transition (row: before, col: after): 
0 
Possible Transition (After): 
0 
fr:3 gene:pol poly_idx:9 HXB2_idx:4932 nuc:T detected:17d
GENE:T4932T AA:F950F NUC:TTT4930-4932TTT
fr:1 gen:NA mut:NA mut_simple:NA mut_AA:NA
fr:2 gen:NA mut:NA mut_simple:NA mut_AA:NA
fr:3 gen:pol mut:TTT4930-4932TTT mut_simple:T4932T mut_AA:F950F

i_raw=10 hxb2=4933 a_WT=G
cdn_bfr:GGA
cdn_afr:---
Detected Codon: before>GGA & after>---
Detected AA: before>G & after>-
Possible Transition (row: before, col: after): 
0 
Possible Transition (After): 
0 
fr:3 gene:pol poly_idx:10 HXB2_idx:4933 nuc:- detected:949d
GENE:G4933- AA:G950- NUC:GGA4933-4935---
fr:1 gen:NA mut:NA mut_simple:NA mut_AA:NA
fr:2 gen:NA mut:NA mut_simple:NA mut_AA:NA
fr:3 gen:pol mut:GGA4933-4935--- mut_simple:G4933- mut_AA:G950-
cdn_bfr:GGA
cdn_afr:GGA
Detected Codon: before>GGA & after>GGA
Detected AA: before>G & after>G
Possible Transition (row: before

cdn_bfr:GAC
cdn_afr:GAC
Detected Codon: before>GAC & after>GAC
Detected AA: before>D & after>D
Possible Transition (row: before, col: after): 
0 
Possible Transition (After): 
0 
fr:3 gene:pol poly_idx:18 HXB2_idx:4941 nuc:C detected:17d
GENE:C4941C AA:D953D NUC:GAC4939-4941GAC
fr:1 gen:NA mut:NA mut_simple:NA mut_AA:NA
fr:2 gen:NA mut:NA mut_simple:NA mut_AA:NA
fr:3 gen:pol mut:GAC4939-4941GAC mut_simple:C4941C mut_AA:D953D

i_raw=19 hxb2=4942 a_WT=C
cdn_bfr:CAG
cdn_afr:---
Detected Codon: before>CAG & after>---
Detected AA: before>Q & after>-
Possible Transition (row: before, col: after): 
0 
Possible Transition (After): 
0 
fr:3 gene:pol poly_idx:19 HXB2_idx:4942 nuc:- detected:949d
GENE:C4942- AA:Q953- NUC:CAG4942-4944---
fr:1 gen:NA mut:NA mut_simple:NA mut_AA:NA
fr:2 gen:NA mut:NA mut_simple:NA mut_AA:NA
fr:3 gen:pol mut:CAG4942-4944--- mut_simple:C4942- mut_AA:Q953-
cdn_bfr:CAG
cdn_afr:CAG
Detected Codon: before>CAG & after>CAG
Detected AA: before>Q & after>Q
Possible Transition

cdn_afr:TAC
Detected Codon: before>TAC & after>TAC
Detected AA: before>Y & after>Y
Possible Transition (row: before, col: after): 
0 
Possible Transition (After): 
0 
fr:3 gene:pol poly_idx:29 HXB2_idx:4952 nuc:A detected:17d
GENE:A4952A AA:Y956Y NUC:TAC4951-4953TAC
fr:1 gen:NA mut:NA mut_simple:NA mut_AA:NA
fr:2 gen:NA mut:NA mut_simple:NA mut_AA:NA
fr:3 gen:pol mut:TAC4951-4953TAC mut_simple:A4952A mut_AA:Y956Y
cdn_bfr:TAC
cdn_afr:TGC
Detected Codon: before>TAC & after>TGC
Detected AA: before>Y & after>C
Possible Transition (row: before, col: after): 
1 
Possible Transition (After): 
0 
fr:3 gene:pol poly_idx:29 HXB2_idx:4952 nuc:G detected:1305d
GENE:A4952G AA:Y956C NUC:TAC4951-4953TGC
fr:1 gen:NA mut:NA mut_simple:NA mut_AA:NA
fr:2 gen:NA mut:NA mut_simple:NA mut_AA:NA
fr:3 gen:pol mut:TAC4951-4953TGC mut_simple:A4952G mut_AA:Y956C

i_raw=30 hxb2=4953 a_WT=C
cdn_bfr:TAC
cdn_afr:---
Detected Codon: before>TAC & after>---
Detected AA: before>Y & after>-
Possible Transition (row: befo

cdn_bfr:GGA
cdn_afr:GGG
Detected Codon: before>GGA & after>GGG
Detected AA: before>G & after>G
Possible Transition (row: before, col: after): 
1 
Possible Transition (After): 
0 
fr:3 gene:pol poly_idx:36 HXB2_idx:4959 nuc:G detected:445d
GENE:A4959G AA:G959G NUC:GGA4957-4959GGG
fr:1 gen:NA mut:NA mut_simple:NA mut_AA:NA
fr:2 gen:NA mut:NA mut_simple:NA mut_AA:NA
fr:3 gen:pol mut:GGA4957-4959GGG mut_simple:A4959G mut_AA:G959G

i_raw=37 hxb2=4960 a_WT=A
cdn_bfr:AAG
cdn_afr:---
Detected Codon: before>AAG & after>---
Detected AA: before>K & after>-
Possible Transition (row: before, col: after): 
0 
Possible Transition (After): 
0 
fr:3 gene:pol poly_idx:37 HXB2_idx:4960 nuc:- detected:949d
GENE:A4960- AA:K959- NUC:AAG4960-4962---
fr:1 gen:NA mut:NA mut_simple:NA mut_AA:NA
fr:2 gen:NA mut:NA mut_simple:NA mut_AA:NA
fr:3 gen:pol mut:AAG4960-4962--- mut_simple:A4960- mut_AA:K959-
cdn_bfr:AAG
cdn_afr:AAG
Detected Codon: before>AAG & after>AAG
Detected AA: before>K & after>K
Possible Transitio

...

cdn_bfr:CTT
cdn_afr:CTT
Detected Codon: before>CTT & after>CTT
Detected AA: before>L & after>L
Possible Transition (row: before, col: after): 
0 
Possible Transition (After): 
0 
fr:3 gene:vpr poly_idx:793 HXB2_idx:5716 nuc:C detected:17d
GENE:C5716C AA:L53L NUC:CTT5716-5718CTT
fr:1 gen:NA mut:NA mut_simple:NA mut_AA:NA
fr:2 gen:NA mut:NA mut_simple:NA mut_AA:NA
fr:3 gen:vpr mut:CTT5716-5718CTT mut_simple:C5716C mut_AA:L53L

i_raw=794 hxb2=5717 a_WT=T
cdn_bfr:CTT,CGT,CCT
cdn_afr:---
Detected Codon: before>CTT/CGT/CCT & after>---
Detected AA: before>L/R/P & after>-
Possible Transition (row: before, col: after): 
0 
0 
0 
Possible Transition (After): 
0 
fr:3 gene:vpr poly_idx:794 HXB2_idx:5717 nuc:- detected:1305d
GENE:T/G/C5717- AA:L/R/P53- NUC:CTT/CGT/CCT5716-5718---
fr:1 gen:NA mut:NA mut_simple:NA mut_AA:NA
fr:2 gen:NA mut:NA mut_simple:NA mut_AA:NA
fr:3 gen:vpr mut:CTT/CGT/CCT5716-5718--- mut_simple:T/G/C5717- mut_AA:L/R/P53-
cdn_bfr:CTT
cdn_afr:CCT
Detected Codon: before>CTT & aft

cdn_afr:TGG
Detected Codon: before>TGG & after>TGG
Detected AA: before>W & after>W
Possible Transition (row: before, col: after): 
0 
Possible Transition (After): 
0 
fr:3 gene:vpr poly_idx:801 HXB2_idx:5724 nuc:G detected:17d
GENE:G5724G AA:W56W NUC:TGG5722-5724TGG
fr:1 gen:NA mut:NA mut_simple:NA mut_AA:NA
fr:2 gen:NA mut:NA mut_simple:NA mut_AA:NA
fr:3 gen:vpr mut:TGG5722-5724TGG mut_simple:G5724G mut_AA:W56W

i_raw=802 hxb2=5725 a_WT=G
cdn_bfr:GAG
cdn_afr:---
Detected Codon: before>GAG & after>---
Detected AA: before>E & after>-
Possible Transition (row: before, col: after): 
0 
Possible Transition (After): 
0 
fr:3 gene:vpr poly_idx:802 HXB2_idx:5725 nuc:- detected:1305d
GENE:G5725- AA:E56- NUC:GAG5725-5727---
fr:1 gen:NA mut:NA mut_simple:NA mut_AA:NA
fr:2 gen:NA mut:NA mut_simple:NA mut_AA:NA
fr:3 gen:vpr mut:GAG5725-5727--- mut_simple:G5725- mut_AA:E56-
cdn_bfr:GAG
cdn_afr:GAG
Detected Codon: before>GAG & after>GAG
Detected AA: before>E & after>E
Possible Transition (row: befor

cdn_bfr:CTA
cdn_afr:---
Detected Codon: before>CTA & after>---
Detected AA: before>L & after>-
Possible Transition (row: before, col: after): 
0 
Possible Transition (After): 
0 
fr:3 gene:vpr poly_idx:811 HXB2_idx:5734 nuc:- detected:1305d
GENE:C5734- AA:L59- NUC:CTA5734-5736---
fr:1 gen:NA mut:NA mut_simple:NA mut_AA:NA
fr:2 gen:NA mut:NA mut_simple:NA mut_AA:NA
fr:3 gen:vpr mut:CTA5734-5736--- mut_simple:C5734- mut_AA:L59-
cdn_bfr:CTA
cdn_afr:CTA
Detected Codon: before>CTA & after>CTA
Detected AA: before>L & after>L
Possible Transition (row: before, col: after): 
0 
Possible Transition (After): 
0 
fr:3 gene:vpr poly_idx:811 HXB2_idx:5734 nuc:C detected:17d
GENE:C5734C AA:L59L NUC:CTA5734-5736CTA
fr:1 gen:NA mut:NA mut_simple:NA mut_AA:NA
fr:2 gen:NA mut:NA mut_simple:NA mut_AA:NA
fr:3 gen:vpr mut:CTA5734-5736CTA mut_simple:C5734C mut_AA:L59L

i_raw=812 hxb2=5735 a_WT=T
cdn_bfr:CTA
cdn_afr:---
Detected Codon: before>CTA & after>---
Detected AA: before>L & after>-
Possible Transition

Excessive output truncated after 524351 bytes.

cdn_afr:TAC
Detected Codon: before>TAA & after>TAC
Detected AA: before>* & after>Y
Possible Transition (row: before, col: after): 
1 
Possible Transition (After): 
0 
fr:3 gene:vpr poly_idx:819 HXB2_idx:5742 nuc:C detected:32d
GENE:A5742C AA:*62Y NUC:TAA5740-5742TAC
fr:1 gen:NA mut:NA mut_simple:NA mut_AA:NA
fr:2 gen:NA mut:NA mut_simple:NA mut_AA:NA
fr:3 gen:vpr mut:TAA5740-5742TAC mut_simple:A5742C mut_AA:*62Y

i_raw=820 hxb2=5743 a_WT=G
cdn_bfr:GAA
cdn_afr:---
Detected Codon: before>GAA & after>---
Detected AA: before>E & after>-
Possible Transition (row: before, col: after): 
0 
Possible Transition (After): 
0 
fr:3 gene:vpr poly_idx:820 HXB2_idx:5743 nuc:- detected:1305d
GENE:G5743- AA:E62- NUC:GAA5743-5745---
fr:1 gen:NA mut:NA mut_simple:NA mut_AA:NA
fr:2 gen:NA mut:NA mut_simple:NA mut_AA:NA
fr:3 gen:vpr mut:GAA5743-5745--- mut_simple:G5743- mut_AA:E62-
cdn_bfr:GAA,CAA
cdn_afr:AAA
Detected Codon: before>GAA/CAA & after>AAA
Detected AA: before>E/Q & after>K
Possible Transition (

## Check the entories of resistant mutations

In [25]:
# Check the ordering is the same as the *-3-poly and *-3-index. 
for i in 1:length(csv_selection.HXB2_index)
    i_HXB2 = csv_selection.HXB2_index[i]
    i_HXB2_v2 = mutant_hxb2[i]
    nuc = csv_selection.nucleotide[i]
    nuc_v2 = mutant_nuc[i]
    @assert (i_HXB2 == i_HXB2_v2 && nuc == nuc_v2)
end

# --- Check whether mutations have been reported or not. ---# 
reported_mutant_data = copy(String.(csv_raw_resist_mut_DH270[:, 1]))
bool_resist_mut_DH270 = [check_mutant_is_in_reported(my_replace_nothing(string(x)), reported_mutant_data) for x in mutant_types_set_AA[3]]
@show sum(bool_resist_mut_DH270)

reported_mutant_data = copy(String.(csv_raw_resist_mut_DH272[:, 1]))
bool_resist_mut_DH272 = [check_mutant_is_in_reported(my_replace_nothing(string(x)), reported_mutant_data) for x in mutant_types_set_AA[3]]
@show sum(bool_resist_mut_DH272)

reported_mutant_data = copy(String.(csv_raw_resist_mut_DH475[:, 1]))
bool_resist_mut_DH475 = [check_mutant_is_in_reported(my_replace_nothing(string(x)), reported_mutant_data) for x in mutant_types_set_AA[3]]
@show sum(bool_resist_mut_DH475)

reported_mutant_data = copy(String.(csv_raw_resist_strain_specific_Abs_CH848[:, 1]))
bool_resist_strain_specific_Abs_CH848 = [check_mutant_is_in_reported(my_replace_nothing(string(x)), reported_mutant_data) for x in mutant_types_set_AA[3]]
@show sum(bool_resist_strain_specific_Abs_CH848)

reported_mutant_data = copy(String.(csv_raw_common_mut_SHIV_CH848[:, 1]))
bool_common_mut_SHIV_CH848 = [check_mutant_is_in_reported(my_replace_nothing(string(x)), reported_mutant_data) for x in mutant_types_set_AA[3]]
@show sum(bool_common_mut_SHIV_CH848);
reversion_true_false = csv_selection.consensus .== csv_selection.nucleotide .!= csv_selection.TF;

sum(bool_resist_mut_DH270) = 33
sum(bool_resist_mut_DH272) = 10
sum(bool_resist_mut_DH475) = 7
sum(bool_resist_strain_specific_Abs_CH848) = 14
sum(bool_common_mut_SHIV_CH848) = 2


In [26]:

# --- Just filling basic information ----#
idx_sort_MPL = sortperm(csv_selection.s_MPL, rev=true);
idx = (mutant_types_set_nuc[3][idx_sort_MPL] .!= "") .* (mutant_date_found[idx_sort_MPL] .> 0)
df = DataFrame(
    HXB2=vec_nuc_idx[idx_sort_MPL][idx], 
)

df[!, "i_AA_fr1"]=replace_nothing_with_missing(vec_AA_idx[1])[idx_sort_MPL][idx]
df[!, "i_AA_fr2"]=replace_nothing_with_missing(vec_AA_idx[2])[idx_sort_MPL][idx]
df[!, "i_AA_fr3"]=replace_nothing_with_missing(vec_AA_idx[3])[idx_sort_MPL][idx]
df[!, "gene_fr1"]=replace_nothing_with_missing(gene_set[1])[idx_sort_MPL][idx]
df[!, "gene_fr2"]=replace_nothing_with_missing(gene_set[2])[idx_sort_MPL][idx]
df[!, "gene_fr3"]=replace_nothing_with_missing(gene_set[3])[idx_sort_MPL][idx]

# -- Just copying fields already in the *-3-poly.csv
for header_temp in names(csv_selection)
    if(!occursin(r"f_at", header_temp) && !occursin(r"epitope", header_temp))
        df[!, header_temp] = csv_selection[:, Symbol(header_temp)][idx_sort_MPL][idx]
    end
end
# --- Writing computed variables including types of mutaions ----#
df[!, "detected_date"] = mutant_date_found[idx_sort_MPL][idx]
#df[!, "mutants_nuc_filtered"] = mutant_types_set_nuc_simple[3][idx_sort_MPL][idx]

df[!, "mutants_nuc"] = mutant_nuc_simple_filtered[idx_sort_MPL][idx]
df[!, "mutants_AA_fr1"] = mutant_types_set_AA_filtered_v2[1][idx_sort_MPL][idx]
df[!, "mutants_AA_fr2"] = mutant_types_set_AA_filtered_v2[2][idx_sort_MPL][idx]
df[!, "mutants_AA_fr3"] = mutant_types_set_AA_filtered_v2[3][idx_sort_MPL][idx]
df[!, "LoopD"] = vec_LoopD[idx_sort_MPL][idx]
df[!, "CD4BS"] = vec_CD4contact_binding[idx_sort_MPL][idx]
df[!, "V1"] = vec_V1[idx_sort_MPL][idx]
df[!, "V2"] = vec_V2[idx_sort_MPL][idx]
df[!, "V3"] = vec_V3[idx_sort_MPL][idx]
df[!, "V4"] = vec_V4[idx_sort_MPL][idx]
df[!, "V5"] = vec_V5[idx_sort_MPL][idx]
df[!, "DH270BS"] = vec_DH270_binding[idx_sort_MPL][idx]
df[!, "DH272BS"] = vec_DH272_binding[idx_sort_MPL][idx]
df[!, "DH475BS"] = vec_DH475_binding[idx_sort_MPL][idx]
df[!, "resist_mut_DH270"] = bool_resist_mut_DH270[idx_sort_MPL][idx]
df[!, "resist_mut_DH272"] = bool_resist_mut_DH272[idx_sort_MPL][idx]
df[!, "resist_mut_DH475"] = bool_resist_mut_DH475[idx_sort_MPL][idx]
df[!, "resist_strain_specific_Abs_CH848"] = bool_resist_strain_specific_Abs_CH848[idx_sort_MPL][idx]
df[!, "common_mut_SHIV_CH848"] = bool_common_mut_SHIV_CH848[idx_sort_MPL][idx]
df[!, "reversion"] = reversion_true_false[idx_sort_MPL][idx]
df[!, "N_linked_glycan_plus_fr1"] = plus_glycan_set[1][idx_sort_MPL][idx]
df[!, "N_linked_glycan_minus_fr1"] = minus_glycan_set[1][idx_sort_MPL][idx]
df[!, "N_linked_glycan_shift_fr1"] = shifted_glycan_set[1][idx_sort_MPL][idx]
df[!, "N_linked_glycan_plus_fr2"] = plus_glycan_set[2][idx_sort_MPL][idx]
df[!, "N_linked_glycan_minus_fr2"] = minus_glycan_set[2][idx_sort_MPL][idx]
df[!, "N_linked_glycan_shift_fr2"] = shifted_glycan_set[2][idx_sort_MPL][idx]
df[!, "N_linked_glycan_plus_fr3"] = plus_glycan_set[3][idx_sort_MPL][idx]
df[!, "N_linked_glycan_minus_fr3"] = minus_glycan_set[3][idx_sort_MPL][idx]
df[!, "N_linked_glycan_shift_fr3"] = shifted_glycan_set[3][idx_sort_MPL][idx]

# -- Just copying single site trajectories *-3-poly.csv
for header_temp in names(csv_selection)
    if(occursin(r"f_at", header_temp) )
        df[!, header_temp] = csv_selection[:, Symbol(header_temp)][idx_sort_MPL][idx]
    end
end
CSV.write("../out/703010848-3-poly_sorted_with_mutant.csv", df);

# CSV files for Rhesus Macaques
- Note, I need to use results of coaligned sequences to compare with CH505 and RMs. 

In [28]:
fname_key_RMs = ["RM6163", "RM6167", "RM6700", "RM6713", "RM6714", "RM6720"];

In [32]:
for key_RM in fname_key_RMs
    @sprintf("%s\n", key_RM)
    csv_selection = DataFrame(CSV.File("../data/mpl/SHIV.CH848/" * key_RM * "-3-poly.csv"));

    gene_set = [[] for _ in 1:3]; vec_AA_idx = [[] for _ in 1:3]; HXB2_set = [[] for _ in 1:3]; 
    vec_nuc_idx = []; frame_set = []; 
    for x in csv_selection.HXB2_index
        num_nuc = get_num_nuc(x)
        frame, gene = index2frame(num_nuc)
        num_nuc, num_AA, gene = map_numNUC_to_numAA(x, frame)
        #@assert(gene==gene_check)

        push!(frame_set, frame)
        push!(vec_nuc_idx, num_nuc)
        for fr in 1:3 
            push!(gene_set[fr], gene[fr])
            push!(vec_AA_idx[fr], num_AA[fr])
            push!(HXB2_set[fr], HXB2(num_nuc, num_AA[fr], gene[fr]))
        end
    end;

    @printf("Fr.1: gene: %s\n", join(unique(gene_set[1]), ", "));
    @printf("Fr.2: gene: %s\n", join(unique(gene_set[2]), ", "));
    @printf("Fr.3: gene: %s\n", join(unique(gene_set[3]), ", "));
    @printf("max: %d, %d\n", maximum(skipmissing(vec_nuc_idx)), maximum(skipmissing(vec_nuc_idx))) 
    @printf("min: %d, %d\n", minimum(skipmissing(vec_nuc_idx)), minimum(skipmissing(vec_nuc_idx)));

    # Use frame 3 where Env gene belongs. 
    vec_DH270_binding = get_true_false_variable_region(vec_AA_idx[3], csv_raw_DH270_binding)
    vec_DH272_binding = get_true_false_variable_region(vec_AA_idx[3], csv_raw_DH272_binding)
    vec_DH475_binding = get_true_false_variable_region(vec_AA_idx[3], csv_raw_DH475_binding)

    vec_CD4contact_binding = get_true_false_variable_region(vec_AA_idx[3], csv_raw_CD4contact)
    vec_V1 = get_true_false_variable_region(vec_AA_idx[3], csv_raw_V1_HV)
    vec_V2 = get_true_false_variable_region(vec_AA_idx[3], csv_raw_V2_HV)
    vec_V3 = get_true_false_variable_region(vec_AA_idx[3], csv_raw_V3_HV)
    vec_V4 = get_true_false_variable_region(vec_AA_idx[3], csv_raw_V4_HV)
    vec_V5 = get_true_false_variable_region(vec_AA_idx[3], csv_raw_V5_HV)
    vec_LoopD = get_true_false_variable_region(vec_AA_idx[3], csv_raw_LoopD);
    # ----------------------------------------------------------------#

    # These are all waat I need. 
    csv_index_and_TF= DataFrame(CSV.File("../data/mpl/SHIV.CH848/" *key_RM* "-3-index.csv"));
    seq_num_raw = readdlm("../data/mpl/SHIV.CH848/" *key_RM* "-3-poly-seq2state.dat", Int);

    # ---- Get mutations ------#
    (mutant_hxb2, mutant_nuc, mutant_date_found, mutant_gene, mutant_types_set_AA, 
    mutant_types_set_nuc, mutant_types_set_nuc_simple, plus_glycan_set, minus_glycan_set, shifted_glycan_set
    ) = Make_combination_of_mutations_with_genetic_background_w_glycan(csv_index_and_TF, seq_num_raw, hxb2csv); 
    mutant_types_set_AA_filtered = []
    push!(mutant_types_set_AA_filtered, filtering_mutations_AA(mutant_types_set_AA[1]));
    push!(mutant_types_set_AA_filtered, filtering_mutations_AA(mutant_types_set_AA[2]));
    push!(mutant_types_set_AA_filtered, filtering_mutations_AA(mutant_types_set_AA[3]));
    mutant_nuc_simple_filtered = filter_nuc_mut(mutant_types_set_nuc_simple);
    # ----------------------------------------------------------------#
    # Check the ordering is the same as the *-3-poly and *-3-index. 
    
    for i in 1:length(csv_selection.HXB2_index)
        i_HXB2 = csv_selection.HXB2_index[i]
        i_HXB2_v2 = mutant_hxb2[i]
        nuc = csv_selection.nucleotide[i]
        nuc_v2 = mutant_nuc[i]
    end

    # --- Check whether mutations have been reported or not. ---# 
    reported_mutant_data = copy(String.(csv_raw_resist_mut_DH270[:, 1]))
    bool_resist_mut_DH270 = [check_mutant_is_in_reported(my_replace_nothing(string(x)), reported_mutant_data) for x in mutant_types_set_AA[3]]
    @show sum(bool_resist_mut_DH270)

    reported_mutant_data = copy(String.(csv_raw_resist_mut_DH272[:, 1]))
    bool_resist_mut_DH272 = [check_mutant_is_in_reported(my_replace_nothing(string(x)), reported_mutant_data) for x in mutant_types_set_AA[3]]
    @show sum(bool_resist_mut_DH272)

    reported_mutant_data = copy(String.(csv_raw_resist_mut_DH475[:, 1]))
    bool_resist_mut_DH475 = [check_mutant_is_in_reported(my_replace_nothing(string(x)), reported_mutant_data) for x in mutant_types_set_AA[3]]
    @show sum(bool_resist_mut_DH475)

    reported_mutant_data = copy(String.(csv_raw_resist_strain_specific_Abs_CH848[:, 1]))
    bool_resist_strain_specific_Abs_CH848 = [check_mutant_is_in_reported(my_replace_nothing(string(x)), reported_mutant_data) for x in mutant_types_set_AA[3]]
    @show sum(bool_resist_strain_specific_Abs_CH848)

    reported_mutant_data = copy(String.(csv_raw_common_mut_SHIV_CH848[:, 1]))
    bool_common_mut_SHIV_CH848 = [check_mutant_is_in_reported(my_replace_nothing(string(x)), reported_mutant_data) for x in mutant_types_set_AA[3]]
    @show sum(bool_common_mut_SHIV_CH848);
    reversion_true_false = csv_selection.consensus .== csv_selection.nucleotide .!= csv_selection.TF;    
    # --- Just filling basic information ----#
    idx_sort_MPL = sortperm(csv_selection.s_MPL, rev=true);
    idx = (mutant_types_set_nuc[3][idx_sort_MPL] .!= "") .* (mutant_date_found[idx_sort_MPL] .> 0)
    df = DataFrame(HXB2=vec_nuc_idx[idx_sort_MPL][idx], )
    
    df[!, "i_AA_fr1"]=replace_nothing_with_missing(vec_AA_idx[1])[idx_sort_MPL][idx]
    df[!, "i_AA_fr2"]=replace_nothing_with_missing(vec_AA_idx[2])[idx_sort_MPL][idx]
    df[!, "i_AA_fr3"]=replace_nothing_with_missing(vec_AA_idx[3])[idx_sort_MPL][idx]
    df[!, "gene_fr1"]=replace_nothing_with_missing(gene_set[1])[idx_sort_MPL][idx]
    df[!, "gene_fr2"]=replace_nothing_with_missing(gene_set[2])[idx_sort_MPL][idx]
    df[!, "gene_fr3"]=replace_nothing_with_missing(gene_set[3])[idx_sort_MPL][idx]

    # -- Just copying fields already in the *-3-poly.csv
    for header_temp in names(csv_selection)
        if(!occursin(r"f_at", header_temp) && !occursin(r"epitope", header_temp))
            df[!, header_temp] = csv_selection[:, Symbol(header_temp)][idx_sort_MPL][idx]
        end
    end

    # --- Writing computed variables including types of mutaions ----#
    df[!, "detected_date"] = mutant_date_found[idx_sort_MPL][idx]
#    df[!, "mutants_nuc_filtered"] = mutant_types_set_nuc_simple[3][idx_sort_MPL][idx]
    df[!, "mutants_nuc_filtered"] = mutant_nuc_simple_filtered[idx_sort_MPL][idx]
    df[!, "mutants_AA_fr1"] = mutant_types_set_AA_filtered[1][idx_sort_MPL][idx]
    df[!, "mutants_AA_fr2"] = mutant_types_set_AA_filtered[2][idx_sort_MPL][idx]
    df[!, "mutants_AA_fr3"] = mutant_types_set_AA_filtered[3][idx_sort_MPL][idx]
    df[!, "LoopD"] = vec_LoopD[idx_sort_MPL][idx]
    df[!, "CD4BS"] = vec_CD4contact_binding[idx_sort_MPL][idx]
    df[!, "V1"] = vec_V1[idx_sort_MPL][idx]
    df[!, "V2"] = vec_V2[idx_sort_MPL][idx]
    df[!, "V3"] = vec_V3[idx_sort_MPL][idx]
    df[!, "V4"] = vec_V4[idx_sort_MPL][idx]
    df[!, "V5"] = vec_V5[idx_sort_MPL][idx]    
    df[!, "DH270BS"] = vec_DH270_binding[idx_sort_MPL][idx]
    df[!, "DH272BS"] = vec_DH272_binding[idx_sort_MPL][idx]
    df[!, "DH475BS"] = vec_DH475_binding[idx_sort_MPL][idx]
    df[!, "resist_mut_DH270"] = bool_resist_mut_DH270[idx_sort_MPL][idx]
    df[!, "resist_mut_DH272"] = bool_resist_mut_DH272[idx_sort_MPL][idx]
    df[!, "resist_mut_DH475"] = bool_resist_mut_DH475[idx_sort_MPL][idx]
    df[!, "resist_strain_specific_Abs_CH848"] = bool_resist_strain_specific_Abs_CH848[idx_sort_MPL][idx]
    df[!, "common_mut_SHIV_CH848"] = bool_common_mut_SHIV_CH848[idx_sort_MPL][idx]
    df[!, "reversion"] = reversion_true_false[idx_sort_MPL][idx]
    df[!, "N_linked_glycan_plus_fr1"] = plus_glycan_set[1][idx_sort_MPL][idx]
    df[!, "N_linked_glycan_minus_fr1"] = minus_glycan_set[1][idx_sort_MPL][idx]
    df[!, "N_linked_glycan_shift_fr1"] = shifted_glycan_set[1][idx_sort_MPL][idx]
    df[!, "N_linked_glycan_plus_fr2"] = plus_glycan_set[2][idx_sort_MPL][idx]
    df[!, "N_linked_glycan_minus_fr2"] = minus_glycan_set[2][idx_sort_MPL][idx]
    df[!, "N_linked_glycan_shift_fr2"] = shifted_glycan_set[2][idx_sort_MPL][idx]
    df[!, "N_linked_glycan_plus_fr3"] = plus_glycan_set[3][idx_sort_MPL][idx]
    df[!, "N_linked_glycan_minus_fr3"] = minus_glycan_set[3][idx_sort_MPL][idx]
    df[!, "N_linked_glycan_shift_fr3"] = shifted_glycan_set[3][idx_sort_MPL][idx]
    
    # -- Just copying single site trajectories *-3-poly.csv
    for header_temp in names(csv_selection)
        if(occursin(r"f_at", header_temp) )
            df[!, header_temp] = csv_selection[:, Symbol(header_temp)][idx_sort_MPL][idx]
        end
    end

    CSV.write("../out/" *key_RM* "-3-poly_sorted_with_mutant.csv", df);
end

Fr.1: gene: 
Fr.2: gene: vpu, 
Fr.3: gene: env
max: 8372, 8372
min: 6233, 6233

i_raw=9 hxb2=6233 a_WT=G
cdn_bfr:GAT
cdn_afr:AAT
Detected Codon: before>GAT & after>AAT
Detected AA: before>D & after>N
Possible Transition (row: before, col: after): 
1 
Possible Transition (After): 
0 
fr:2 gene:vpu poly_idx:9 HXB2_idx:6233 nuc:A detected:70d
GENE:G6233A AA:D58N NUC:GAT6233-6235AAT
cdn_bfr:GTG
cdn_afr:GTA
Detected Codon: before>GTG & after>GTA
Detected AA: before>V & after>V
Possible Transition (row: before, col: after): 
1 
Possible Transition (After): 
0 
fr:3 gene:env poly_idx:9 HXB2_idx:6233 nuc:A detected:70d
GENE:G6233A AA:V3V NUC:GTG6231-6233GTA
fr:1 gen:NA mut:NA mut_simple:NA mut_AA:NA
fr:2 gen:vpu mut:GAT6233-6235AAT mut_simple:G6233A mut_AA:D58N
fr:3 gen:env mut:GTG6231-6233GTA mut_simple:G6233A mut_AA:V3V
cdn_bfr:GAT
cdn_afr:GAT
Detected Codon: before>GAT & after>GAT
Detected AA: before>D & after>D
Possible Transition (row: before, col: after): 
0 
Possible Transition (After):


i_raw=75 hxb2=6290 a_WT=T
cdn_bfr:TTG
cdn_afr:CTG
Detected Codon: before>TTG & after>CTG
Detected AA: before>L & after>L
Possible Transition (row: before, col: after): 
1 
Possible Transition (After): 
0 
fr:2 gene:vpu poly_idx:75 HXB2_idx:6290 nuc:C detected:70d
GENE:T6290C AA:L77L NUC:TTG6290-6292CTG
cdn_bfr:TTT
cdn_afr:TTC
Detected Codon: before>TTT & after>TTC
Detected AA: before>F & after>F
Possible Transition (row: before, col: after): 
1 
Possible Transition (After): 
0 
fr:3 gene:env poly_idx:75 HXB2_idx:6290 nuc:C detected:70d
GENE:T6290C AA:F22F NUC:TTT6288-6290TTC
fr:1 gen:NA mut:NA mut_simple:NA mut_AA:NA
fr:2 gen:vpu mut:TTG6290-6292CTG mut_simple:T6290C mut_AA:L77L
fr:3 gen:env mut:TTT6288-6290TTC mut_simple:T6290C mut_AA:F22F
cdn_bfr:TTG
cdn_afr:TTG
Detected Codon: before>TTG & after>TTG
Detected AA: before>L & after>L
Possible Transition (row: before, col: after): 
0 
Possible Transition (After): 
0 
fr:2 gene:vpu poly_idx:75 HXB2_idx:6290 nuc:T detected:0d
GENE:T6290T

fr:3 gen:env mut:ACT/ACA6375-6377ACC mut_simple:T/A6377C mut_AA:T51T
cdn_bfr:ACT
cdn_afr:ACT
Detected Codon: before>ACT & after>ACT
Detected AA: before>T & after>T
Possible Transition (row: before, col: after): 
0 
Possible Transition (After): 
0 
fr:3 gene:env poly_idx:162 HXB2_idx:6377 nuc:T detected:0d
GENE:T6377T AA:T51T NUC:ACT6375-6377ACT
fr:1 gen:NA mut:NA mut_simple:NA mut_AA:NA
fr:2 gen:NA mut:NA mut_simple:NA mut_AA:NA
fr:3 gen:env mut:ACT6375-6377ACT mut_simple:T6377T mut_AA:T51T

i_raw=183 hxb2=6398 a_WT=C
cdn_bfr:GCC
cdn_afr:GCC
Detected Codon: before>GCC & after>GCC
Detected AA: before>A & after>A
Possible Transition (row: before, col: after): 
0 
Possible Transition (After): 
0 
fr:3 gene:env poly_idx:183 HXB2_idx:6398 nuc:C detected:0d
GENE:C6398C AA:A58A NUC:GCC6396-6398GCC
fr:1 gen:NA mut:NA mut_simple:NA mut_AA:NA
fr:2 gen:NA mut:NA mut_simple:NA mut_AA:NA
fr:3 gen:env mut:GCC6396-6398GCC mut_simple:C6398C mut_AA:A58A
cdn_bfr:GCC
cdn_afr:GCT
Detected Codon: before>GC

0 
fr:3 gene:env poly_idx:220 HXB2_idx:6435 nuc:- detected:0d
GENE:-6435- AA:71- NUC:6435-6437---
fr:1 gen:NA mut:NA mut_simple:NA mut_AA:NA
fr:2 gen:NA mut:NA mut_simple:NA mut_AA:NA
fr:3 gen:env mut:6435-6437--- mut_simple:-6435- mut_AA:71-
cdn_bfr:ACA
cdn_afr:ACA
Detected Codon: before>ACA & after>ACA
Detected AA: before>T & after>T
Possible Transition (row: before, col: after): 
0 
Possible Transition (After): 
0 
fr:3 gene:env poly_idx:220 HXB2_idx:6435 nuc:A detected:0d
GENE:A6435A AA:T71T NUC:ACA6435-6437ACA
fr:1 gen:NA mut:NA mut_simple:NA mut_AA:NA
fr:2 gen:NA mut:NA mut_simple:NA mut_AA:NA
fr:3 gen:env mut:ACA6435-6437ACA mut_simple:A6435A mut_AA:T71T

i_raw=221 hxb2=6436 a_WT=C
cdn_bfr:---
cdn_afr:---
Detected Codon: before>--- & after>---
Detected AA: before>- & after>-
Possible Transition (row: before, col: after): 
0 
Possible Transition (After): 
0 
fr:3 gene:env poly_idx:221 HXB2_idx:6436 nuc:- detected:0d
GENE:-6436- AA:71- NUC:6435-6437---
fr:1 gen:NA mut:NA mut_simpl

cdn_bfr:TGT
cdn_afr:TGT
Detected Codon: before>TGT & after>TGT
Detected AA: before>C & after>C
Possible Transition (row: before, col: after): 
0 
Possible Transition (After): 
0 
fr:3 gene:env poly_idx:231 HXB2_idx:6446 nuc:T detected:0d
GENE:T6446T AA:C74C NUC:TGT6444-6446TGT
fr:1 gen:NA mut:NA mut_simple:NA mut_AA:NA
fr:2 gen:NA mut:NA mut_simple:NA mut_AA:NA
fr:3 gen:env mut:TGT6444-6446TGT mut_simple:T6446T mut_AA:C74C

i_raw=232 hxb2=6447 a_WT=G
cdn_bfr:GTA
cdn_afr:---
Detected Codon: before>GTA & after>---
Detected AA: before>V & after>-
Possible Transition (row: before, col: after): 
0 
Possible Transition (After): 
0 
fr:3 gene:env poly_idx:232 HXB2_idx:6447 nuc:- detected:252d
GENE:G6447- AA:V75- NUC:GTA6447-6449---
fr:1 gen:NA mut:NA mut_simple:NA mut_AA:NA
fr:2 gen:NA mut:NA mut_simple:NA mut_AA:NA
fr:3 gen:env mut:GTA6447-6449--- mut_simple:G6447- mut_AA:V75-
cdn_bfr:GTA
cdn_afr:GTA
Detected Codon: before>GTA & after>GTA
Detected AA: before>V & after>V
Possible Transition (

...

fr:3 gen:env mut:TAT6243-6245TGT mut_simple:A6244G mut_AA:Y7C

i_raw=33 hxb2=6248 a_WT=A
cdn_bfr:ACA
cdn_afr:ACA
Detected Codon: before>ACA & after>ACA
Detected AA: before>T & after>T
Possible Transition (row: before, col: after): 
0 
Possible Transition (After): 
0 
fr:2 gene:vpu poly_idx:33 HXB2_idx:6248 nuc:A detected:0d
GENE:A6248A AA:T63T NUC:ACA6248-6250ACA
cdn_bfr:CCA
cdn_afr:CCA
Detected Codon: before>CCA & after>CCA
Detected AA: before>P & after>P
Possible Transition (row: before, col: after): 
0 
Possible Transition (After): 
0 
fr:3 gene:env poly_idx:33 HXB2_idx:6248 nuc:A detected:0d
GENE:A6248A AA:P8P NUC:CCA6246-6248CCA
fr:1 gen:NA mut:NA mut_simple:NA mut_AA:NA
fr:2 gen:vpu mut:ACA6248-6250ACA mut_simple:A6248A mut_AA:T63T
fr:3 gen:env mut:CCA6246-6248CCA mut_simple:A6248A mut_AA:P8P
cdn_bfr:ACA
cdn_afr:GCA
Detected Codon: before>ACA & after>GCA
Detected AA: before>T & after>A
Possible Transition (row: before, col: after): 
1 
Possible Transition (After): 
0 
fr:2 gene:v

fr:2 gen:vpu mut:TTG6290-6292GTG mut_simple:T6290G mut_AA:L77V
fr:3 gen:env mut:TTT6288-6290TTG mut_simple:T6290G mut_AA:F22L
cdn_bfr:TTG
cdn_afr:TTG
Detected Codon: before>TTG & after>TTG
Detected AA: before>L & after>L
Possible Transition (row: before, col: after): 
0 
Possible Transition (After): 
0 
fr:2 gene:vpu poly_idx:75 HXB2_idx:6290 nuc:T detected:0d
GENE:T6290T AA:L77L NUC:TTG6290-6292TTG
cdn_bfr:TTT
cdn_afr:TTT
Detected Codon: before>TTT & after>TTT
Detected AA: before>F & after>F
Possible Transition (row: before, col: after): 
0 
Possible Transition (After): 
0 
fr:3 gene:env poly_idx:75 HXB2_idx:6290 nuc:T detected:0d
GENE:T6290T AA:F22F NUC:TTT6288-6290TTT
fr:1 gen:NA mut:NA mut_simple:NA mut_AA:NA
fr:2 gen:vpu mut:TTG6290-6292TTG mut_simple:T6290T mut_AA:L77L
fr:3 gen:env mut:TTT6288-6290TTT mut_simple:T6290T mut_AA:F22F

i_raw=79 hxb2=6294 a_WT=A
cdn_bfr:GAT
cdn_afr:GAT
Detected Codon: before>GAT & after>GAT
Detected AA: before>D & after>D
Possible Transition (row: bef

fr:2 gen:vpu mut:TTG6305-6307TTG mut_simple:T6305T mut_AA:L82L
fr:3 gen:env mut:ATT6303-6305ATT mut_simple:T6305T mut_AA:I27I

i_raw=95 hxb2=6310 a_WT=A
cdn_bfr:TAA
cdn_afr:TAA
Detected Codon: before>TAA & after>TAA
Detected AA: before>* & after>*
Possible Transition (row: before, col: after): 
0 
Possible Transition (After): 
0 
fr:2 gene:vpu poly_idx:95 HXB2_idx:6310 nuc:A detected:0d
GENE:A6310A AA:*84* NUC:TAA6308-6310TAA
cdn_bfr:AAT
cdn_afr:AAT
Detected Codon: before>AAT & after>AAT
Detected AA: before>N & after>N
Possible Transition (row: before, col: after): 
0 
Possible Transition (After): 
0 
fr:3 gene:env poly_idx:95 HXB2_idx:6310 nuc:A detected:0d
GENE:A6310A AA:N29N NUC:AAT6309-6311AAT
fr:1 gen:NA mut:NA mut_simple:NA mut_AA:NA
fr:2 gen:vpu mut:TAA6308-6310TAA mut_simple:A6310A mut_AA:*84*
fr:3 gen:env mut:AAT6309-6311AAT mut_simple:A6310A mut_AA:N29N
cdn_bfr:TAA
cdn_afr:TAG
Detected Codon: before>TAA & after>TAG
Detected AA: before>* & after>*
Possible Transition (row: bef

Excessive output truncated after 524347 bytes.

cdn_bfr:GCA
cdn_afr:GCA
Detected Codon: before>GCA & after>GCA
Detected AA: before>A & after>A
Possible Transition (row: before, col: after): 
0 
Possible Transition (After): 
0 
fr:3 gene:env poly_idx:151 HXB2_idx:6366 nuc:G detected:0d
GENE:G6366G AA:A48A NUC:GCA6366-6368GCA
fr:1 gen:NA mut:NA mut_simple:NA mut_AA:NA
fr:2 gen:NA mut:NA mut_simple:NA mut_AA:NA
fr:3 gen:env mut:GCA6366-6368GCA mut_simple:G6366G mut_AA:A48A

i_raw=156 hxb2=6371 a_WT=A
cdn_bfr:AAA
cdn_afr:AAA
Detected Codon: before>AAA & after>AAA
Detected AA: before>K & after>K
Possible Transition (row: before, col: after): 
0 
Possible Transition (After): 
0 
fr:3 gene:env poly_idx:156 HXB2_idx:6371 nuc:A detected:0d
GENE:A6371A AA:K49K NUC:AAA6369-6371AAA
fr:1 gen:NA mut:NA mut_simple:NA mut_AA:NA
fr:2 gen:NA mut:NA mut_simple:NA mut_AA:NA
fr:3 gen:env mut:AAA6369-6371AAA mut_simple:A6371A mut_AA:K49K
cdn_bfr:AAA
cdn_afr:AAG
Detected Codon: before>AAA & after>AAG
Detected AA: before>K & after>K
Possible Transition (ro

# Processes for Jointed Rhessu

In [35]:
f_dir = "../data/mpl/SHIV.CH848/"
csv_index_and_TF = DataFrame(CSV.File(f_dir * "Marged-RMs-index_with_poly_idx.csv"));
s_MPL_RMs = readdlm(f_dir * "RMs-poly-seq2state-MPL.dat")
s_SL_RMs = readdlm(f_dir * "RMs-poly-seq2state-SL.dat");
dx_RMs = readdlm(f_dir * "numerator-RMs-3-poly-seq2state.dat");
fname_key_RMs_and_Human = ["703010848", "RM6163", "RM6167", "RM6700", "RM6713", "RM6714", "RM6720"];
n_max_RMs = length(fname_key_RMs_and_Human);

## Get processed mutations for jointed RMs for CSV file 
(polymorphic_index_global, HXB2_index_global, nucleotide_global, 
    TF_global, TF_global_RMs, nonsynonymous_global, nonsynonymous_global_RMs, 
    edge_gap_global, edge_gap_global_RMs, exposed_global, exposed_global_RMs, 
    flanking_global, flanking_global_RMs, glycan_global, glycan_global_RMs, consensus_global, consensus_global_RMs, 
    dx_marginalized, s_MPL_marginalized, s_SL_marginalized, s_MPL_global, s_SL_global) = get_csv_RMs(
    f_dir, csv_index_and_TF, s_MPL_RMs, s_SL_RMs, dx_RMs, fname_key_RMs_and_Human);

In [36]:
df = DataFrame( 
    polymorphic=polymorphic_index_global, 
    HXB2=HXB2_index_global, 
    nuc=nucleotide_global, 
    TF=TF_global, 
    TF_RMs=TF_global_RMs,
    nonsynonymous=nonsynonymous_global, 
    nonsynonymous_RMs=nonsynonymous_global_RMs, 
    consensus=consensus_global, 
    consensus_RMs=consensus_global_RMs,
    edge_gap=edge_gap_global, 
    edge_gap_RMs=edge_gap_global_RMs, 
    exposed=exposed_global, 
    exposed_RMs=exposed_global_RMs, 
    flanking=flanking_global, 
    flanking_RMs=flanking_global_RMs, 
    glycan=glycan_global, 
    glycan_RMs=glycan_global_RMs,  
)

df[:, "dx_RMs"] = dx_marginalized
df[:, "s_MPL_RMs"] = s_MPL_marginalized
for k in 1:n_max_RMs
    df[:, "s_MPL_"*fname_key_RMs_and_Human[k]] = s_MPL_global[k]
end
df[:, "s_SL_RMs"] = s_SL_marginalized
for k in 1:n_max_RMs
    df[:, "s_SL_"*fname_key_RMs_and_Human[k]] = s_SL_global[k]
end
CSV.write("../out/Marged-RMs-3-poly_SHIVCH848.csv", df);

## Normalizing selection coefficients 

In [37]:
csv_in = DataFrame(CSV.File("../out/Marged-RMs-3-poly_SHIVCH848.csv"));

polymorphic_unique = sort(unique(csv_in.polymorphic))
poly_max = maximum(polymorphic_unique)+1
s_mpl_TF = [0.0 for _ in 1:poly_max]
s_sl_TF = [0.0 for _ in 1:poly_max]
NA_list = [false for _ in 1:poly_max]
for x in polymorphic_unique
    tf_in = custom_mode(csv_in.TF_RMs[csv_in.polymorphic .== x])
    if(tf_in != "NA")
        idx1 = csv_in.polymorphic .== x
        idx2 = csv_in.nuc .== tf_in
        idx = idx1 .* idx2
        s_mpl_TF[x+1] = csv_in.s_MPL_RMs[idx][1]
        s_sl_TF[x+1] = csv_in.s_SL_RMs[idx][1]
        NA_list[x+1] = true
    end
end;

s_MPL_norm = []; s_SL_norm = []
for i in 1:length(csv_in.polymorphic)
    if( NA_list[ csv_in.polymorphic[i] + 1 ] )
        s_mpl_temp = csv_in.s_MPL_RMs[i] - s_mpl_TF[ csv_in.polymorphic[i] + 1 ]
        s_sl_temp = csv_in.s_SL_RMs[i] - s_sl_TF[ csv_in.polymorphic[i] + 1 ]
        push!(s_MPL_norm, s_mpl_temp)
        push!(s_SL_norm, s_sl_temp)
    else
        push!(s_MPL_norm, "NA")
        push!(s_SL_norm, "NA")
    end
end;

In [38]:
idx_non_NA = csv_in.TF_RMs .!= "NA"
df = DataFrame( 
    polymorphic=csv_in.polymorphic[idx_non_NA], 
    HXB2=csv_in.HXB2[idx_non_NA], 
    nuc=csv_in.nuc[idx_non_NA], 
    TF=csv_in.TF[idx_non_NA], 
    TF_RMs=csv_in.TF_RMs[idx_non_NA],
    nonsynonymous=csv_in.nonsynonymous[idx_non_NA], 
    nonsynonymous_RMs=csv_in.nonsynonymous_RMs[idx_non_NA], 
    consensus=csv_in.consensus[idx_non_NA], 
    consensus_RMs=csv_in.consensus_RMs[idx_non_NA],
    edge_gap=csv_in.edge_gap[idx_non_NA], 
    edge_gap_RMs=csv_in.edge_gap_RMs[idx_non_NA], 
    exposed=csv_in.exposed[idx_non_NA], 
    exposed_RMs=csv_in.exposed_RMs[idx_non_NA], 
    flanking=csv_in.flanking[idx_non_NA], 
    flanking_RMs=csv_in.flanking_RMs[idx_non_NA], 
    glycan=csv_in.glycan[idx_non_NA], 
    glycan_RMs=csv_in.glycan_RMs[idx_non_NA],  
)

df[:, "dx_RMs"] = csv_in.dx_RMs[idx_non_NA]
df[:, "s_MPL_RMs"] = s_MPL_norm[idx_non_NA]
for k in 1:n_max_RMs
    df[:, "s_MPL_"*fname_key_RMs_and_Human[k]] = csv_in[idx_non_NA, Symbol("s_MPL_"*fname_key_RMs_and_Human[k])]
end
df[:, "s_SL_RMs"] = s_SL_norm[idx_non_NA]
for k in 1:n_max_RMs
    df[:, "s_SL_"*fname_key_RMs_and_Human[k]] = csv_in[idx_non_NA, Symbol("s_SL_"*fname_key_RMs_and_Human[k])]
end
CSV.write("../out/Marged-RMs-normalized-3-poly_SHIVCH848.csv", df);



# Making CSV file containing mutations for marginalized RMs.  

Note: this process consider only Env gene;

In [39]:
csv_selection = DataFrame(CSV.File("../out/Marged-RMs-normalized-3-poly_SHIVCH848.csv"));
csv_index_and_TF= DataFrame(CSV.File("../data/mpl/SHIV.CH848/Marged-RMs-index_with_poly_idx.csv"));

In [42]:
(vec_gene_idx, vec_nuc_idx, vec_AA_idx, vec_TF, 
    vec_DH270_binding, vec_DH272_binding, vec_DH475_binding, vec_CD4contact_binding, 
    vec_V1, vec_V2, vec_V3, vec_V4, vec_V5, vec_LoopD, 
    mutant_types_set_nuc_simple, mutant_types_set_AA_filtered, 
    Nlinked_plus_set, Nlinked_minus_set, Nlinked_shift_set,
    bool_resist_mut_DH270, bool_resist_mut_DH272, bool_resist_mut_DH475, 
    bool_resist_strain_specific_Abs_CH848, 
    bool_common_mut_SHIV_CH848, reversion_true_false) = get_jointed_RMs_for_CSV_SHIV848(
    csv_selection, csv_index_and_TF, fname_key_RMs_and_Human, fname_key_RMs);
  

gene: env
max: 8375, 8375
min: 6225, 6225
Processing 703010848, done.
Processing RM6163, done.
Processing RM6167, done.
Processing RM6700, done.
Processing RM6713, done.
Processing RM6714, done.
Processing RM6720, done.


In [43]:
# --- Just filling basic information ----#
idx_sort_MPL = sortperm(csv_selection.s_MPL_RMs, rev=true);
idx = mutant_types_set_nuc_simple[idx_sort_MPL] .!= ""
df = DataFrame(
    HXB2=vec_nuc_idx[idx_sort_MPL][idx], 
    AA_idx=replace_nothing_with_missing(vec_AA_idx)[idx_sort_MPL][idx],
    gene=replace_nothing_with_missing(vec_gene_idx)[idx_sort_MPL][idx]
)
df[!, "mutants_nuc"] = mutant_types_set_nuc_simple[idx_sort_MPL][idx]
df[!, "mutants_AA"] = mutant_types_set_AA_filtered[idx_sort_MPL][idx]
# -- Just copying fields already in the *-3-poly.csv
for header_temp in names(csv_selection)
    df[!, header_temp] = csv_selection[:, Symbol(header_temp)][idx_sort_MPL][idx]
end
for k in 1:length(fname_key_RMs_and_Human)
    df[:, "N_plus_"*fname_key_RMs_and_Human[k]] = Nlinked_plus_set[k][idx_sort_MPL][idx]
    df[:, "N_minus_"*fname_key_RMs_and_Human[k]] = Nlinked_minus_set[k][idx_sort_MPL][idx]
    df[:, "N_shift_"*fname_key_RMs_and_Human[k]] = Nlinked_shift_set[k][idx_sort_MPL][idx]
end

# --- Writing computed variables including types of mutaions ----#
df[!, "LoopD"] = vec_LoopD[idx_sort_MPL][idx]
df[!, "CD4BS"] = vec_CD4contact_binding[idx_sort_MPL][idx]
df[!, "V1"] = vec_V1[idx_sort_MPL][idx]
df[!, "V2"] = vec_V2[idx_sort_MPL][idx]
df[!, "V3"] = vec_V3[idx_sort_MPL][idx]
df[!, "V4"] = vec_V4[idx_sort_MPL][idx]
df[!, "V5"] = vec_V5[idx_sort_MPL][idx]
df[!, "DH270BS"] = vec_DH270_binding[idx_sort_MPL][idx]
df[!, "DH272BS"] = vec_DH272_binding[idx_sort_MPL][idx]
df[!, "DH475BS"] = vec_DH475_binding[idx_sort_MPL][idx]
df[!, "resist_mut_DH270"] = bool_resist_mut_DH270[idx_sort_MPL][idx]
df[!, "resist_mut_DH272"] = bool_resist_mut_DH272[idx_sort_MPL][idx]
df[!, "resist_mut_DH475"] = bool_resist_mut_DH475[idx_sort_MPL][idx]
df[!, "resist_strain_specific_Abs_CH848"] = bool_resist_strain_specific_Abs_CH848[idx_sort_MPL][idx]
df[!, "common_mut_SHIV_CH848"] = bool_common_mut_SHIV_CH848[idx_sort_MPL][idx]
df[!, "reversion"] = reversion_true_false[idx_sort_MPL][idx];
CSV.write("../out/RMs-merged-3-poly_sorted_with_mutant_CH848.csv", df);

## Enrichment  -> CSV 

In [13]:
#csv_raw_CH848 = DataFrame(CSV.File("../out/703010848-3-poly_sorted_with_mutant.csv"));
csv_raw_CH848 = DataFrame(CSV.File("/net/dali/home/barton/kais/HIV_coevolution_analysis/out/redo_on_cluster/CSV/CH848/703010848-3-poly_sorted_with_mutant.csv"));

In [39]:
#"""
#csv_raw_CH848 = DataFrame(CSV.File("../out/703010848-3-poly_sorted_with_mutant.csv"));
csv_raw_CH848 = DataFrame(CSV.File("/net/dali/home/barton/kais/HIV_coevolution_analysis/out/redo_on_cluster/CSV/CH848/703010848-3-poly_sorted_with_mutant.csv"));
categories = ["Rev.", "V1", "V2", "V3", "V4", "V5", "LD", "CD4"];
#s_threshold_RMs = 0.02
s_threshold_RMs = 0.022
idx_significant = csv_raw_CH848.s_MPL .>= s_threshold_RMs;

len_tot_entries = length(csv_raw_CH848.HXB2_index)
len_selected = count(idx_significant)
α_selected = len_selected / len_tot_entries;
@printf("α_selectd = %.2f %%\n", 100*α_selected);

(n_subjected_common, n_estimated_common, n_tot_common, x_fold_common) = get_x_fold(csv_raw_CH848.common_mut_SHIV_CH848, idx_significant)
(n_subjected_strain_specific_Abs, n_estimated_strain_specific_Abs, n_tot_strain_specific_Abs, x_fold_strain_specific_Abs) = get_x_fold(csv_raw_CH848.resist_strain_specific_Abs_CH848, idx_significant)
idx_strain_specific_general = csv_raw_CH848.common_mut_SHIV_CH848 .|| csv_raw_CH848.resist_strain_specific_Abs_CH848
(n_subjected_strain_specific_gen, n_estimated_strain_specific_gen, n_tot_strain_specific_gen, x_fold_strain_specific_gen) = get_x_fold(idx_strain_specific_general, idx_significant)

(n_subjected_LoopD, n_estimated_LoopD, n_tot_LoopD, x_fold_LoopD) = get_x_fold(csv_raw_CH848.LoopD, idx_significant)
(n_subjected_CD4BS, n_estimated_CD4BS, n_tot_CD4BS, x_fold_CD4BS) = get_x_fold(csv_raw_CH848.CD4BS, idx_significant)
(n_subjected_V1, n_estimated_V1, n_tot_V1, x_fold_V1) = get_x_fold(csv_raw_CH848.V1, idx_significant)
(n_subjected_V2, n_estimated_V2, n_tot_V2, x_fold_V2) = get_x_fold(csv_raw_CH848.V2, idx_significant)
(n_subjected_V3, n_estimated_V3, n_tot_V3, x_fold_V3) = get_x_fold(csv_raw_CH848.V3, idx_significant)
(n_subjected_V4, n_estimated_V4, n_tot_V4, x_fold_V4) = get_x_fold(csv_raw_CH848.V4, idx_significant)
(n_subjected_V5, n_estimated_V5, n_tot_V5, x_fold_V5) = get_x_fold(csv_raw_CH848.V5, idx_significant)
(n_subjected_reversion, n_estimated_reversion, n_tot_reversion, x_fold_reversion) = get_x_fold(csv_raw_CH848.reversion, idx_significant) ;

values = [x_fold_reversion, x_fold_V1, x_fold_V2, x_fold_V3, x_fold_V4, x_fold_V5, x_fold_LoopD, 
     x_fold_CD4BS]
df = DataFrame(
    categories=categories,
    enrichment=[@sprintf("%.4f", x) for x in values]
)
CSV.write("../out/enrichment_CH848.csv", df);
#"""

α_selectd = 0.96 %


In [40]:
csv_raw_CH848 = DataFrame(CSV.File("/net/dali/home/barton/kais/HIV_coevolution_analysis/out/redo_on_cluster/CSV/CH848/703010848-3-poly_sorted_with_mutant.csv"));
categories = ["Rev.", "V1", "V2", "V3", "V4", "V5", "LD", "CD4"];

s_threshold_RMs_set = collect(0.015:0.001:0.04)
x_fold_V1_set = []
x_fold_V2_set = []
x_fold_V3_set = []
x_fold_V4_set = []
x_fold_V5_set = []
x_fold_LoopD_set = []
α_selected_est = []
for s_threshold_RMs in s_threshold_RMs_set
    idx_significant = csv_raw_CH848.s_MPL .>= s_threshold_RMs;
    len_tot_entries = length(csv_raw_CH848.HXB2_index)
    len_selected = count(idx_significant)
    α_selected = len_selected / len_tot_entries;
    #@printf("s_threshold_RMs = %.3f , α_selectd = %.2f %%\n", s_threshold_RMs, 100*α_selected);

    (n_subjected_common, n_estimated_common, n_tot_common, x_fold_common) = get_x_fold(csv_raw_CH848.common_mut_SHIV_CH848, idx_significant)
    (n_subjected_strain_specific_Abs, n_estimated_strain_specific_Abs, n_tot_strain_specific_Abs, x_fold_strain_specific_Abs) = get_x_fold(csv_raw_CH848.resist_strain_specific_Abs_CH848, idx_significant)
    idx_strain_specific_general = csv_raw_CH848.common_mut_SHIV_CH848 .|| csv_raw_CH848.resist_strain_specific_Abs_CH848
    (n_subjected_strain_specific_gen, n_estimated_strain_specific_gen, n_tot_strain_specific_gen, x_fold_strain_specific_gen) = get_x_fold(idx_strain_specific_general, idx_significant)

    (n_subjected_LoopD, n_estimated_LoopD, n_tot_LoopD, x_fold_LoopD) = get_x_fold(csv_raw_CH848.LoopD, idx_significant)
    (n_subjected_V1, n_estimated_V1, n_tot_V1, x_fold_V1) = get_x_fold(csv_raw_CH848.V1, idx_significant)
    (n_subjected_V2, n_estimated_V2, n_tot_V2, x_fold_V2) = get_x_fold(csv_raw_CH848.V2, idx_significant)
    (n_subjected_V3, n_estimated_V3, n_tot_V3, x_fold_V3) = get_x_fold(csv_raw_CH848.V3, idx_significant)
    (n_subjected_V4, n_estimated_V4, n_tot_V4, x_fold_V4) = get_x_fold(csv_raw_CH848.V4, idx_significant)
    (n_subjected_V5, n_estimated_V5, n_tot_V5, x_fold_V5) = get_x_fold(csv_raw_CH848.V5, idx_significant)

    push!(x_fold_V1_set, x_fold_V1)
    push!(x_fold_V2_set, x_fold_V2)
    push!(x_fold_V3_set, x_fold_V3)
    push!(x_fold_V4_set, x_fold_V4)
    push!(x_fold_V5_set, x_fold_V5)
    push!(x_fold_LoopD_set, x_fold_LoopD)
    push!(α_selected_est, α_selected)
    @printf("s = %.3f , α_selectd = %.2f %% EV1 = %.3f  EV2 = %.3f \n", s_threshold_RMs, 100*α_selected, x_fold_V1, x_fold_V2);
end




s = 0.015 , α_selectd = 2.47 % EV1 = 3.289  EV2 = 0.487 
s = 0.016 , α_selectd = 2.27 % EV1 = 3.046  EV2 = 0.530 
s = 0.017 , α_selectd = 2.03 % EV1 = 2.671  EV2 = 0.594 
s = 0.018 , α_selectd = 1.71 % EV1 = 2.643  EV2 = 0.705 
s = 0.019 , α_selectd = 1.57 % EV1 = 2.682  EV2 = 0.766 
s = 0.020 , α_selectd = 1.39 % EV1 = 2.386  EV2 = 0.867 
s = 0.021 , α_selectd = 1.19 % EV1 = 2.783  EV2 = 0.675 
s = 0.022 , α_selectd = 0.96 % EV1 = 3.446  EV2 = 0.418 
s = 0.023 , α_selectd = 0.87 % EV1 = 3.808  EV2 = 0.000 
s = 0.024 , α_selectd = 0.76 % EV1 = 3.552  EV2 = 0.000 
s = 0.025 , α_selectd = 0.66 % EV1 = 4.131  EV2 = 0.000 
s = 0.026 , α_selectd = 0.60 % EV1 = 4.554  EV2 = 0.000 
s = 0.027 , α_selectd = 0.56 % EV1 = 4.267  EV2 = 0.000 
s = 0.028 , α_selectd = 0.53 % EV1 = 4.511  EV2 = 0.000 
s = 0.029 , α_selectd = 0.49 % EV1 = 4.934  EV2 = 0.000 
s = 0.030 , α_selectd = 0.46 % EV1 = 5.263  EV2 = 0.000 
s = 0.031 , α_selectd = 0.43 % EV1 = 4.934  EV2 = 0.000 
s = 0.032 , α_selectd = 0.41 % 

In [34]:
color_set = ["blue", "green", "orange", "purple", "yellow", "cyan"]
p1 = Plots.plot(100*s_threshold_RMs_set, x_fold_V1_set, label="V1", 
    xlabel=L"Threshold by selection coefficients $\hat{s}$", 
    ylabel="Enrichment ratio", 
    foreground_color_legend = nothing,
    c = color_set[1],
    labelfontsize=12,
    legendfontsize=12, 
    m=:circle,
    margin=3mm,
    legend=:false,
    tickfontsize=12,
)
Plots.plot!(100*s_threshold_RMs_set, x_fold_V2_set, label="V2", m=:circle, c = color_set[2])
Plots.plot!(100*s_threshold_RMs_set, x_fold_V3_set, label="V3", m=:circle, c = color_set[3],)
Plots.plot!(100*s_threshold_RMs_set, x_fold_V4_set, label="V4", m=:circle, c = color_set[4],)
Plots.plot!(100*s_threshold_RMs_set, x_fold_V5_set, label="V5", m=:circle, c = color_set[5],)
Plots.plot!(100*s_threshold_RMs_set, x_fold_LoopD_set, label="LoopD", m=:circle, c = color_set[6])

p2 = Plots.plot(100*α_selected_est, ones(size(α_selected_est)), label=:false, c=:gray)
Plots.plot!(100*α_selected_est, x_fold_V1_set, label="V1", 
    xlabel=L"Threshold by fraction top  $x$ %", 
    ylabel="Enrichment ratio", 
    c = color_set[1],
    foreground_color_legend = nothing,
    labelfontsize=12,
    legendfontsize=12, 
    m=:circle,
    tickfontsize=12,
    margin=3mm,
    legend=(0.8, 0.99)
)
Plots.plot!(100*α_selected_est, x_fold_V2_set, label="V2", m=:circle, c = color_set[2],)
Plots.plot!(100*α_selected_est, x_fold_V3_set, label="V3", m=:circle, c = color_set[3],)
Plots.plot!(100*α_selected_est, x_fold_V4_set, label="V4", m=:circle, c = color_set[4],)
Plots.plot!(100*α_selected_est, x_fold_V5_set, label="V5", m=:circle, c = color_set[5],)
Plots.plot!(100*α_selected_est, x_fold_LoopD_set, label="LoopD", m=:circle, c = color_set[6],)

Plots.plot(p1,p2, layout=(1,2), size=(700, 350))
Plots.savefig("../fig/enrichment_vs_threshold_CH848.pdf")

"/net/dali/home/barton/kais/submission/HIV_Coevolution/fig/enrichment_vs_threshold_CH848.pdf"

6552